# 0. 이론 - 로지스틱 회귀

## 1. 이진 로지스틱 회귀 시그모이드 사용 이유

합격을 1, 불합격을 0이라고 하였을 때 그래프를 그려보면 아래와 같습니다.

![](images/로지스틱.png)


이러한 점들을 표현하는 그래프는 알파벳의 S자 형태로 표현됩니다. 
이러한  x와 y의 관계를 표현하기 위해서는 직선을 표현하는 함수가 아니라 S자 형태로 표현할 수 있는 함수가 필요합니다. 직선을 사용할 경우 보통 분류 작업이 제대로 동작하지 않습니다.  

이번 예제의 경우 실제값. 즉, 레이블에 해당하는 **y가 0 또는 1**이라는 두 가지 값만을 가지므로, 이 문제를 풀기 위해서 **예측값은 0과 1사이의 값**을 가지도록 합니다. 0과 1사이의 값을 확률로 해석하면 문제를 풀기가 훨씬 용이해집니다.  
**최종 예측값이 0.5보다 작으면 0으로 예측했다고 판단하고, 0.5보다 크면 1로 예측**했다고 판단합니다. 
만약 y=wx+b의 직선을 사용할 경우, y값이 음의 무한대부터 양의 무한대와 같은 큰 수들도 가질 수 있는데 이는 직선이 분류 문제에 적합하지 않은 두번째 이유입니다.

출력이 0과 1사이의 값을 가지면서 S자 형태로 그려지는 함수로 시그모이드 함수(Sigmoid function)가 있습니다.

## 2. 이진 로지스틱 회귀

- 이진 로지스틱 회귀 (for 2개의 클래스 분류 문제)
    - 목적변수가 A라면 1, B라면 0으로 고려
    - 주어진 피처 x에 대해서, A가 될 확률을 p라고 하면 B가 될 확률은 1-p 
    - 임의의 x에 대해서 p가 0.5보다 크면 A로 분류, 작으면 B로 분류
    
- 이진 로지스틱 회귀에서 주어진 피처 x에 대해서 p를 얻어내기 위해서 y를 어떻게 고려해야 할까?  
![](images/로시.png)
    - **가설함수는 선형함수를 통해서 로짓 (logit)을 얻고, 시그모이드 (sigmoid)함수를 적용** 
        - (로짓을 통해 –무한대~+무한대 범위 -> 시그모이드 적용하여 0~1 범위로)
        - 가설함수 = p (p: 1로 분류될 확률, 1-p: 0으로 분류될 확률)
        - 로짓함수, 시그모이드함수
            ![](images/로짓.png)![](images/시그.png)

    




## 3. 비용함수 - 교차 엔트로피


- h: y가 1로 분류될 확률로 0~1 사이의 값을 가짐 (릿지+시그모이드)
- y: 0 또는 1의 값
- 좋은 가설함수 h란 h와 y가 비슷한 값을 가지는 것!!! -> 교차 엔트로피 사용

![](images/코스트.png)

![](images/이진비용.png)

## 4. 경사하강법

![](images/이진경사.png)

# 1. 시그모이드, 가설, 비용, 경사하강법 정의

In [20]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
# 시그모이드 함수
def sigmoid(z):
    return 1/(1+np.exp(z))

In [22]:
# 가설 함수 (릿지+시그모이드)
def hypothesis_function(x,theta):
    z = np.dot(-x,theta)    # 릿지 ((381*4)*(4*1)=(381*1))
    return sigmoid(z)       # 릿지 + 시그모이드

In [23]:
# 비용 함수
def compute_cost(x,y,theta):
    m = y.shape[0]  # 데이터 수
    J = (-1/m)*y.T.dot(np.log(hypothesis_function(x,theta)))+(1-y).T.dot(np.log(1-hypothesis_function(x,theta)))
    return J

In [24]:
def minimize_gradient(x,y,theta,iterations=10000,alpha=0.01):
    m     = y.size
    Cost  = []  # 100번 업데이트마다 비용
    Theta = []  # 하이퍼 파라미터
    
    for I in range(iterations):
        original_theta = theta
        for i in range(theta.size): # 4번 반복
            partial_marginal = x[:,i].reshape(-1,1)# x중 한 피처 -> 하나의 열벡터로!(381*1)
            delta            = hypothesis_function(x,original_theta) - y # (381*1)
            # 그래디언트 정의 (걍 받아들이기)
            grad_i           = delta.T.dot(partial_marginal) # ((1*381)*(381*1)=1*1)
            # 경사하강법으로 theta 업데이트
            theta[i]         = theta[i] - alpha*grad_i
        if I%100 ==0:
            Theta.append(theta)
            Cost.append(compute_cost(x,y,theta))
    return theta,np.array(Cost),np.array(Theta)
            

# 2. Breast Cancer 데이터에 logistic 회귀 적용

In [25]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True, as_frame = False)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=1234)
X_train, X_test = X_train[:, :3], X_test[:, :3]
# reshape 필수
y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [26]:
######## 1. 전처리 - 표준 스케일링
scaler = StandardScaler()
scaler.fit(X_train) 

X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

######## 1. 전처리 - train, test 데이터에 절편추가
n, n_test = X_train.shape[0], X_test.shape[0]
X_train, X_test = np.append(np.ones((n, 1)), X_train,axis=1), np.append(np.ones((n_test, 1)),X_test,axis=1)

In [27]:
######## 1. 전처리 - 하이퍼 파라미터(w) 초기화
theta = np.ones((X_train.shape[1], 1))
X_train.shape

(381, 4)

In [28]:
######## 2. 경사하강법으로 theta 업데이트
theta, Cost, Theta = minimize_gradient(X_train,y_train,theta)

In [29]:
######## 3. 정확도 측정
y_train_pred = np.where(hypothesis_function(X_train,theta)>0.5,1,0)
y_test_pred  = np.where(hypothesis_function(X_test,theta)>0.5,1,0)
print(f'학습 데이터셋 정확도:{(y_train == y_train_pred).sum() / len(y_train) * 100: .2f}%')
print(f'테스트 데이터셋 정확도:{(y_test == y_test_pred).sum() / len(y_test) * 100: .2f}%')

학습 데이터셋 정확도: 93.18%
테스트 데이터셋 정확도: 87.23%


# 2. LogisticRegression 클래스 사용하기

1번과 달리 간단하게 로지스틱 회귀로 예측할 수 있음! (동일한 방법)

In [16]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33,random_state=1234)

# 절편추가 불필요
X_train = X_train.iloc[:, :3]
X_test  = X_test.iloc[:, :3]

In [19]:
clf = LogisticRegression(random_state=1234, max_iter=1000, C=100)    
# C : 정규화 강도의 역수. 높은 값일수록 정규화가 덜 됨
# solver 옵션(알고리즘)에 따라 다양한 방법으로 진행할 수 있음  

clf          = clf.fit(X_train, y_train)    # 학습
y_train_pred = clf.predict(X_train)         # 훈련 데이터 예측
y_pred       = clf.predict(X_test)          # 테스트 데이터 예측

print(f'학습 데이터셋 정확도:{(y_train == y_train_pred).sum() / len(y_train) * 100: .2f}%')
print(f'테스트 데이터셋 정확도:{(y_test == y_pred).sum() / len(y_test) * 100: .2f}%')

학습 데이터셋 정확도: 93.18%
테스트 데이터셋 정확도: 87.23%


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i